# ETL Pipeline Preparation
### 1. Import libraries and load datasets.

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# load messages dataset
messages = pd.read_csv('../data/messages.csv')
print(messages.shape) # checking the shape of the df
messages.head()

(26248, 4)


,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
# load categories dataset
categories = pd.read_csv('../data/categories.csv')
print(categories.shape) # checking the shape of the df
categories.head()

(26248, 2)


,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.

In [4]:
# merge datasets
df = messages.merge(categories
                   ,how = 'inner'
                   ,on = ['id'])
print(df.shape)                   
df.head()

(26386, 5)


,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.

In [5]:
# create a dataframe of the 36 individual category columns
categories = df['categories'].str.split(';'
                                       ,expand = True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [6]:
# select the first row of the categories dataframe
row = categories.iloc[0, :]
# strip unnecessary characters
category_colnames = row.apply(lambda x: x.rstrip ('-0 1'))
print(category_colnames)

0                    related
1                    request
2                      offer
3                aid_related
4               medical_help
5           medical_products
6          search_and_rescue
7                   security
8                   military
9                child_alone
10                     water
11                      food
12                   shelter
13                  clothing
14                     money
15            missing_people
16                  refugees
17                     death
18                 other_aid
19    infrastructure_related
20                 transport
21                 buildings
22               electricity
23                     tools
24                 hospitals
25                     shops
26               aid_centers
27      other_infrastructure
28           weather_related
29                    floods
30                     storm
31                      fire
32                earthquake
33                      cold
34            

In [8]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.

In [9]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].astype(str).str[-1]    
    # convert column from string to numeric
    categories[column] = pd.to_numeric(categories[column])
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [10]:
# drop the original categories column from `df`
df.drop(['categories']
       ,axis = 1
       ,inplace = True)
df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [11]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories]
              ,axis = 1
              ,sort = False)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [12]:
# check number of duplicate rows
print('There are:', df.duplicated().sum(), 'duplicated rows in the dataframe.')
# check number of duplicate id's
print('There are:', df['id'].duplicated().sum(), 'duplicated id\'s in the dataframe.')
# check number of duplicate messages
print('There are:', df['message'].duplicated().sum(), 'duplicated messages\'s in the dataframe.')


There are: 170 duplicated rows in the dataframe.
There are: 206 duplicated id's in the dataframe.
There are: 209 duplicated messages's in the dataframe.


In [13]:
# drop duplicate rows
df.drop_duplicates(keep = 'first'
                  ,inplace = True)

# drop duplicates with the same id's
df.drop_duplicates(subset = ['id']
                  ,inplace = True)

# drop duplicates with the same messages
df.drop_duplicates(subset = ['message']
                  ,keep = False
                  ,inplace = True)

In [14]:
# check number of duplicates
print('There are:', df.duplicated().sum(), 'duplicated rows in the dataframe.')
print('There are:', df['id'].duplicated().sum(), 'duplicated id\'s in the dataframe.')
print('There are:', df['message'].duplicated().sum(), 'duplicated messages in the dataframe.')

There are: 0 duplicated rows in the dataframe.
There are: 0 duplicated id's in the dataframe.
There are: 0 duplicated messages in the dataframe.


### 7. Check for inconsistencies in all target columns

#### Observations

    * Child_alone column doesn't contain any valuable information
    * Related column contain rows with value == 2
    
#### Decisions on current Observations

    * Drop child_alone column
    * Further study rows where related == 2

In [15]:
categories.max().sort_values() # max values in the categories columns

0
child_alone               0
death                     1
transport                 1
buildings                 1
electricity               1
tools                     1
hospitals                 1
shops                     1
aid_centers               1
other_infrastructure      1
weather_related           1
floods                    1
storm                     1
fire                      1
earthquake                1
cold                      1
infrastructure_related    1
other_aid                 1
direct_report             1
refugees                  1
request                   1
offer                     1
aid_related               1
medical_help              1
medical_products          1
search_and_rescue         1
other_weather             1
security                  1
water                     1
food                      1
shelter                   1
clothing                  1
money                     1
missing_people            1
military                  1
related           

In [16]:
# drop the child_alone column from df
print(df.shape)
df.drop(['child_alone']
       ,axis = 1
       ,inplace = True)
print(df.shape)

(26176, 40)
(26176, 39)


#### Observations
- Most of the messages with related value == 2 are in different languages than english
#### Decisions on current Observations
-  Deciding to drop all rows with related == 2

In [17]:
# checking messages with related values == 2
list(df[df['related'] == 2]['message'])

 citizens live here that all will be ok and that everyone ( could be the world ) hurrying to try to make us survive.",
 'This is my address : Cersine 8 Prolong. .. . ',
 'I would like to have some information on the last disaster',
 "No location : Ou se trouve l'ambassade du Sngal en Haiti",
 "Je veux savoir comment je peux obtenir de l'aide pour mon orphelinat situ  Delmas 31, svp dites-moi ce que je dois faire",
 '142, Ruelle Beaulieu, Mon Repos 44 Carrefour ',
 '. ..  Esdras. Address : Solidarit Village, Toussain Louverture Street 83A!',
 'Route des dalles prolonges #277 after Morne Jean-Pierre, thank you do. ..  ',
 'we are in rue Dessalines Petit goaves and it necessary to..? ',
 'I am very happy to get the messages that tell me about the precautions I have to take. Thank you very much, If you could make it so that food comes my way it would be good because .. ',
 'mouin vle kite ayiti depi kek jou. ',
 'Mwen ta renmen jwenn yon djb. ske gen posiblite pou mwen jwenn li? Kisa ou ka

In [18]:
# deciding to drop rows where related == 2
print(df.shape)
df.drop(df[df['related'] == 2].index
       ,axis = 0
       ,inplace = True)
print(df.shape)

(26176, 39)
(25988, 39)


### 7. Save the clean dataset into an sqlite database.

In [19]:
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df.to_sql('Message_Category'
         ,engine
         ,index = False
         ,if_exists = 'replace')